<a href="https://colab.research.google.com/github/chongzicbo/nlp-ml-dl-notes/blob/master/code/seq2seq/Seq2Seq%E8%AF%A6%E8%A7%A3%E5%8F%8A%E5%AE%9E%E7%8E%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.Seq2Seq是什么？
&emsp;&emsp;Seq2Seq学习是训练一个模型，将来自一个领域的序列(例如英语语句)转换为另一个领域的序列(例如法语)。

```
"the cat sat on the mat" -> [Seq2Seq model] -> "le chat etait assis sur le tapis"
```
&emsp;&emsp;Seq2Seq模型可以用于机器翻译或者自由问答(在给定自然语言问题的情况下生成自然语言答案)。通常，它可以在任何需要生成文本的情况下使用。
&emsp;&emsp;我们可以使用多种方法来处理Seq2Seq任务，比如循环神经网络(RNN)或者(卷积神经网络)。在这里，我们将重点介绍RNN。

### 1.1 输入和输出序列的长度相同

&emsp;&emsp;当输入和输出序列具有相同的长度时，您可以仅使用LSTM或者GRU网络或者它们的堆叠网络来实现此类模型。例如训练一个教RNN学习加法的模型。

![在这里插入图片描述](https://blog.keras.io/img/seq2seq/addition-rnn.png#pic_center)

但是，在通常情况下，我们的输入序列和输出序列的长度并不一样。

### 1.2 输入序列和输出序列长度不同

&emsp;&emsp;一般情况下，输入序列和输出序列具有不同的长度(例如，机器翻译)，并且需要整个输入序列才能开始预测目标。这需要更高级的设置，也就是人们在没有其他上下文情况下提到"序列到序列模型"时通常所指的东西。运行方式如下：
* RNN层(或者RNN的堆叠)作为编码器(encoder)：它处理输入序列并返回其自己的内部状态(internal state)。请注意，我们丢弃了编码器RNN的输出，仅使用其状态(state)。这个状态(state)将作为"上下文"或者"条件"来服务于下一步的decoder。
* 另一个RNN层(或者其堆叠)作为解码器(decoder):在给定目标序列的前一个字符的情况下，对其进行训练以预测目标序列的下一个字符。具体而言，它被训练为将目标序列变成相同的序列，但偏移一个时间步，在这种情况下，称为"teacher forcing"。重要的是，编码器使用来自编码器的状态向量作为初始状态，这就是解码器如何获取有关应该生成的信息的方式。在给定输入序列[...t]的情况，编码器会学习生成目标序列[t+1...]。
![](https://blog.keras.io/img/seq2seq/seq2seq-teacher-forcing.png)

&emsp;&emsp;上述是训练模型，在推理模式中，即当我们想解码未知的输入序列时，我们会经历一个略有不同的过程:
* 1）将输入序列编码为状态向量
* 2）从大小为1的目标序列开始(仅是序列开始字符)
* 3）将状态向量和1个字符的目标序列提供给解码器，以生成下一个字符的预测。
* 4）使用这些预测来采样下一个字符(argmax)。
* 5）将采样的字符追加到目标序列
* 6）重复上述过程直到生成序列结束字符或达到字符数限制。

![](https://blog.keras.io/img/seq2seq/seq2seq-inference.png)
在训练的时候也可以采用此种方式，即通过将解码器的预测重新注入到解码器中。

## 2.keras实现Seq2Seq
* 实现英语到法语的翻译任务
* 编码器和解码器均使用单层LSTM。
* 基于字符级别的序列到序列模型

大概过程如下：
* 1）把输入和输出序列转为3维numpy张量,encoder_input_data, decoder_input_data, decoder_target_data:
* 2）给定encoder_input_data和decoder_input_data，训练一个基础的基于LSTM的Seq2Seq模型来预测decoder_target_data
* 3) 解码测试序列检查模型是否有效

因为训练过程和推理过程(解码序列)完全不同，尽管它们都使用相同的内部层，我们对两者使用不同的模型。







In [30]:
from __future__ import print_function
from tensorflow.keras.models import  Model
from tensorflow.keras.layers import Input,LSTM,Dense
import tensorflow.keras as keras
import numpy as np

In [31]:
#参数设置
batch_size=64
epochs=100
latent_dim=256
num_samples=10000 #样本数量
data_path="/content/drive/My Drive/data/fra-en.txt"

In [32]:
#训练数据准备
input_texts=[]
target_texts=[]
input_characters=set()
target_characters=set()
with open(data_path,"r",encoding="utf-8") as f:
  lines=f.read().split("\n")
for line in lines[:min(num_samples,len(lines)-1)]:
  input_text,target_text=line.split("\t")
  #对于目标序列使用"\t"键作为序列的起始字符
  #使用""\n"作为结束字符
  target_text="\t"+target_text+"\n"
  input_texts.append(input_text)
  target_texts.append(target_text)
  for char in input_text:
    if char not in input_characters:
      input_characters.add(char)
  for char in target_text:
    if char not in target_characters:
      target_characters.add(char)


input_characters=sorted(list(input_characters))
target_characters=sorted(list(target_characters))

num_encoder_tokens=len(input_characters)#所有输入序列去重字符数,区分大小写
num_decoder_tokens=len(target_characters)#所有输出序列去重字符数

max_encoder_seq_length=max([len(txt) for txt in input_texts])
max_decoder_deq_length=max([len(txt) for txt in target_texts])

In [33]:
print("样本数量：",len(input_texts))
print("输入序列字符数:",num_encoder_tokens)
print("输出序列字符数:",num_decoder_tokens)
print("输入序列最长序列长度:",max_encoder_seq_length)
print("输出序列最长序列长度:",max_decoder_deq_length)

样本数量： 10000
输入序列字符数: 70
输出序列字符数: 93
输入序列最长序列长度: 16
输出序列最长序列长度: 59


In [34]:
" ".join(input_characters)

"  ! $ % & ' , - . 0 1 2 3 5 6 7 8 9 : ? A B C D E F G H I J K L M N O P Q R S T U V W Y a b c d e f g h i j k l m n o p q r s t u v w x y z"

In [35]:
input_token_index=dict([(char,i) for i,char in enumerate(input_characters)])
target_token_index=dict([char,i] for i,char in enumerate(target_characters))

In [36]:
input_token_index[" "]

0

In [37]:
encoder_input_data=np.zeros((len(input_texts),max_encoder_seq_length,num_encoder_tokens),dtype="float32")
decoder_input_data=np.zeros((len(input_texts),max_decoder_deq_length,num_decoder_tokens),dtype="float32")
decoder_target_data=np.zeros((len(input_texts),max_decoder_deq_length,num_decoder_tokens),dtype="float32")

for i,(input_text,target_text) in enumerate(zip(input_texts,target_texts)):
  for t,char in enumerate(input_text):
    encoder_input_data[i,t,input_token_index[char]]=1
  encoder_input_data[i,t+1:,input_token_index[" "]]=1  #长度不够的序列使用"  "进行padding

  for t,char in enumerate(target_text):
    decoder_input_data[i,t,target_token_index[char]]=1
    if t>0:
      #目标数据decoder_target_data不包括起始字符，并向前一步(teacher forcing)
      decoder_target_data[i,t-1,target_token_index[char]]=1 
  decoder_input_data[i,t+1:,target_token_index[" "]]=1
  decoder_target_data[i,t:,target_token_index[" "]]=1

In [38]:
encoder_input_data.shape

(10000, 16, 70)

In [39]:
encoder_input_data[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [40]:
#模型训练网络结构搭建

#encoder端
encoder_inputs=Input(shape=(None,num_encoder_tokens))
encoder=LSTM(latent_dim,return_state=True) #return_state参数控制是否返回cell state
encoder_outputs,state_h,state_c=encoder(encoder_inputs)
encoder_states=[state_h,state_c]

#decoder
decoder_inputs=Input(shape=(None,num_decoder_tokens))
#返回所有的output sequences和internal states。在训练过程中不使用return states,但在推理中使用
decoder_lstm=LSTM(latent_dim,return_sequences=True,return_state=True)#return_sequences控制是否返回所有时间步的hidden state
decoder_outputs,_,_=decoder_lstm(decoder_inputs,initial_state=encoder_states) #使用encoder的输出状态作为deocder的初始状态
decoder_dense=Dense(num_decoder_tokens,activation="softmax")
decoder_outputs=decoder_dense(decoder_outputs)

model=Model([encoder_inputs,decoder_inputs],decoder_outputs)

In [41]:
model.compile(optimizer="rmsprop",loss="categorical_crossentropy",metrics=["accuracy"])
model.fit([encoder_input_data,decoder_input_data],decoder_target_data,batch_size=batch_size,epochs=epochs,validation_split=0.2)

Epoch 1/100
125/125 [==============================] - 2s 18ms/step - loss: 1.1857 - accuracy: 0.7244 - val_loss: 1.0744 - val_accuracy: 0.7079
Epoch 2/100
125/125 [==============================] - 1s 12ms/step - loss: 0.8510 - accuracy: 0.7695 - val_loss: 0.8599 - val_accuracy: 0.7620
Epoch 3/100
125/125 [==============================] - 2s 12ms/step - loss: 0.6924 - accuracy: 0.8064 - val_loss: 0.7326 - val_accuracy: 0.7876
Epoch 4/100
125/125 [==============================] - 2s 12ms/step - loss: 0.5997 - accuracy: 0.8261 - val_loss: 0.6669 - val_accuracy: 0.8060
Epoch 5/100
125/125 [==============================] - 2s 14ms/step - loss: 0.5515 - accuracy: 0.8383 - val_loss: 0.6209 - val_accuracy: 0.8188
Epoch 6/100
125/125 [==============================] - 2s 12ms/step - loss: 0.5125 - accuracy: 0.8496 - val_loss: 0.5851 - val_accuracy: 0.8289
Epoch 7/100
125/125 [==============================] - 2s 12ms/step - loss: 0.4826 - accuracy: 0.8576 - val_loss: 0.5636 - val_accuracy:

In [42]:
model.summary()

Model: "functional_21"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_15 (InputLayer)           [(None, None, 70)]   0                                            
__________________________________________________________________________________________________
input_16 (InputLayer)           [(None, None, 93)]   0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 256), (None, 334848      input_15[0][0]                   
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, None, 256),  358400      input_16[0][0]                   
                                                                 lstm_2[0][1]         

### 模型推理
* 编码输入和获取初始的decoder state
* 使用初始state和序列的开始字符作为目标来运行decoder的一步，输出将作为下一个目标字符
* 使用现在的目标字符和状态来重复以上过程


In [43]:
encoder_model=Model(encoder_inputs,encoder_states) #推理模式的encoder端

#推理模式decoder端
decoder_state_input_h=Input(shape=(latent_dim,)) #hidden state
decoder_state_input_c=Input(shape=(latent_dim,)) #cell state 
decoder_states_inputs=[decoder_state_input_h,decoder_state_input_c]
decoder_outputs,state_h,state_c=decoder_lstm(decoder_inputs,initial_state=decoder_states_inputs) #decoder_lstm在训练阶段已经训练好了，推理阶段直接拿来用
decoder_states=[state_h,state_c]
decoder_outputs=decoder_dense(decoder_outputs)#decoder_dense在训练阶段已经训练好了，推理阶段直接拿来用
decoder_model=Model([decoder_inputs]+decoder_states_inputs,[decoder_outputs]+decoder_states)

In [44]:
decoder_model.summary()

Model: "functional_25"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_16 (InputLayer)           [(None, None, 93)]   0                                            
__________________________________________________________________________________________________
input_17 (InputLayer)           [(None, 256)]        0                                            
__________________________________________________________________________________________________
input_18 (InputLayer)           [(None, 256)]        0                                            
__________________________________________________________________________________________________
lstm_3 (LSTM)                   [(None, None, 256),  358400      input_16[0][0]                   
                                                                 input_17[0][0]       

In [45]:
reverse_input_char_index=dict((i,char) for char ,i in input_token_index.items())
reverse_target_char_index=dict((i,char) for char,i in target_token_index.items())

In [46]:
#生成翻译序列

def decode_sequence(input_seq):
  #编码输入作为state向量
  states_value=encoder_model.predict(input_seq)

  target_seq=np.zeros((1,1,num_decoder_tokens)) #长度为1的目标序列
  target_seq[0,0,target_token_index["\t"]]=1

  stop_condition=False
  decoded_sentence=""

  while not stop_condition:
    output_tokens,h,c=decoder_model.predict([target_seq]+states_value)
    sampled_token_index=np.argmax(output_tokens[0,-1,:])
    sampled_char=reverse_target_char_index[sampled_token_index]
    decoded_sentence+=sampled_char

    if(sampled_char=="\n" or len(decoded_sentence)>max_decoder_deq_length):
      stop_condition=True
    target_seq=np.zeros((1,1,num_decoder_tokens))
    target_seq[0,0,sampled_token_index]=1
    states_value=[h,c]
  return decoded_sentence

In [47]:
for seq_index in range(100):
  input_seq=encoder_input_data[seq_index:seq_index+1]
  decoded_sentence=decode_sequence(input_seq)
  print("-")
  print("Input sentence:",input_texts[seq_index])
  print("Decoded sentence:",decoded_sentence)

-
Input sentence: Go.
Decoded sentence: Va !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Hi.
Decoded sentence: Salut !

-
Input sentence: Run!
Decoded sentence: Courez !

-
Input sentence: Run!
Decoded sentence: Courez !

-
Input sentence: Who?
Decoded sentence: Qui ?

-
Input sentence: Wow!
Decoded sentence: Ça alors !

-
Input sentence: Fire!
Decoded sentence: Au feu !

-
Input sentence: Help!
Decoded sentence: À l'aide !

-
Input sentence: Jump.
Decoded sentence: Saute.

-
Input sentence: Stop!
Decoded sentence: Ça suffit !

-
Input sentence: Stop!
Decoded sentence: Ça suffit !

-
Input sentence: Stop!
Decoded sentence: Ça suffit !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: Wait!
Decoded sentence: Attendez !

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Go on.
Decoded sentence: Poursuivez.

-
Input sentence: Hello!
Decoded sentence: Bonjour !

-
